Import librairies

In [135]:
import gspread
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
from matplotlib import rcParams
rcParams['font.family'] = 'MS Gothic'

Connection with google sheets

In [ ]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('../api-key.json', scope)
client = gspread.authorize(creds)

# Open Google Sheets
spreadsheet = client.open_by_key('1tWzHyOUIsNMERChwDtMJlgUURB0aIxNlOkPDp5CDTkk')

Get every tables

In [ ]:
data_sheet = spreadsheet.get_worksheet(1)
hocg_analysis_sheet = spreadsheet.get_worksheet(2)
general_data_sheet = spreadsheet.get_worksheet(5)

data_table = data_sheet.get_all_records()
market_value_table = hocg_analysis_sheet.get('A1:D2')
product_relative_data_table = hocg_analysis_sheet.get('A4:L19')
association_table = hocg_analysis_sheet.get('A23:CN37')
collabs_table = hocg_analysis_sheet.get('A39:C40')
boosters_general_table = general_data_sheet.get('A1:D3')
decks_general_table = general_data_sheet.get('A5:H7')

Generate data frames

In [160]:
df_data = pd.DataFrame(data_table)
# print(df_data)
df_market_value = pd.DataFrame(market_value_table)
# print(df_market_value)
df_product_relative_data = pd.DataFrame(product_relative_data_table)
df_product_relative_data.iloc[0] = df_product_relative_data.iloc[0].replace("", np.nan).ffill()
df_product_relative_data.columns = df_product_relative_data.iloc[1]
df_product_relative_data = df_product_relative_data.drop(1).reset_index(drop=True)
df_product_relative_data = df_product_relative_data.set_index(df_product_relative_data.columns[0])
# print(df_product_relative_data)
df_association = pd.DataFrame(association_table)
# print(df_association)
df_collabs = pd.DataFrame(collabs_table)
# print(df_collabs)
df_boosters_general = pd.DataFrame(boosters_general_table).T
df_boosters_general.columns = df_boosters_general.iloc[0]
df_boosters_general = df_boosters_general.drop(0).reset_index(drop=True)
df_boosters_general['Release date'] = pd.to_datetime(df_boosters_general['Release date'], format='%Y年%m月%d日', errors='coerce')
# print(df_boosters_general)
df_decks_general = pd.DataFrame(decks_general_table).T
df_decks_general.columns = df_decks_general.iloc[0]
df_decks_general = df_decks_general.drop(0).reset_index(drop=True)
df_decks_general['Release date'] = pd.to_datetime(df_decks_general['Release date'], format='%Y年%m月%d日', errors='coerce')
# print(df_decks_general)

In [ ]:
# print(df_product_relative_data)

# Add boosters
product_realease_date = dict(zip(df_boosters_general['Boosters Names'], df_boosters_general['Release date']))
# Add decks
product_realease_date.update(dict(zip(df_decks_general['Decks Names'], df_decks_general['Release date'])))

dates = sorted(list(set(product_realease_date.values())))
dates_string = [date.strftime('%Y年%m月%d日') for date in dates]

date_counter_boosters = {date: 0 for date in dates}
for booster in df_boosters_general['Boosters Names']:
    # Récupérer la date de sortie du booster
    release_date = product_realease_date.get(booster)
    
    if release_date in date_counter_boosters:
        # Incrémenter la valeur associée à cette date
        date_counter_boosters[release_date] += 1

date_counter_decks = {date: 0 for date in dates}
for decks in df_decks_general['Decks Names']:
    # Récupérer la date de sortie du booster
    release_date = product_realease_date.get(decks)
    
    if release_date in date_counter_decks:
        # Incrémenter la valeur associée à cette date
        date_counter_decks[release_date] += 1

values_boosters = list(date_counter_boosters.values())
values_boosters = [np.nan if value == 0 else value for value in values_boosters]

values_decks = list(date_counter_decks.values())
values_decks = [np.nan if value == 0 else value for value in values_decks]

plt.plot(dates_string, values_boosters, marker='o', label="Ligne 1", color='blue', linestyle='-', linewidth=2)
plt.plot(dates_string, values_decks, marker='o', label="Ligne 2", color='green', linestyle='-', linewidth=2)

plt.show()

In [ ]:
labels = ['test1', 'test2', 'test3']

values1 = [20, np.nan, 15]
values2 = [12, 23, np.nan]
values3 = [np.nan, 21, 13]

# Remplir les NaN de values1 par interpolation
values1_interp = np.interp([0, 1, 2], [0, 2], [20, 15]) 

plt.plot(labels, values1_interp, marker='o', label="Ligne 1", color='blue', linestyle='-', markevery=[0, 2], linewidth=2)
plt.plot(labels, values2, marker='o', label="Ligne 2", color='green', linestyle='-', linewidth=2)
plt.plot(labels, values3, marker='o', label="Ligne 3", color='red', linestyle='-', linewidth=2)

# Titre et labels des axes
plt.title("Graphique avec trois lignes et des valeurs manquantes")
plt.xlabel("Tests")
plt.ylabel("Valeurs")

# Affichage du graphique
plt.legend()
plt.show()
